### Introduction notes: 
Why we care about modeling blood flow: 
- blood pressure and blood flow rate affect: Wall shear stress (how much friction blood puts on the vessel wall),  Wall tension (how stretched the vessel wall becomes)
- Both of these are directly tied to diseases like: aneurysms (ballooning of arteries), stenosis (narrowing of arteries), +  other cardiovascular disorders

Different types of blood-flow models
- Lumped parameter models: treat parts of the circulatory system like simple circuits (fast + approximate)
- **1D wave propagation models: treat blood flow along the length of an artery**
    - **They can track pressure waves traveling down vessels (efficiant and clinically useful)**
- 1.5D models: A mix of 1D with some extra features
- 3D models: Full fluid simulations (Navier–Stokes equations) → expensive

Two numerical methods:
Finite Element Method (FEM)
- Uses simple polynomial shape functions (like straight lines or parabolas)
- Needs many small elements to be accurate, especially at high frequencies

**Spectral Element Method (SEM)**
- **Uses shape functions built from exact wave solutions (very accurate)**

\*** Time-domain SEM (uses high-order polynomials with with Newton-Raphson iteration) vs **Frequency-domain SEM (Builds elements from exact frequency- domain wave solutions)**

Viscoelasticity 
- Artery walls are viscoelasctic → store energy (like elastic materials) + dissipate energy (like a dashpot) + they show hysteresis (pressure diameter loops don’t match on the way up vs down) 
- No one had used the spectral element method to study blood flow in viscoelastic arteries

Purpose of the paper
- Develop a 1D nonlinear spectral element model for blood flow in viscoelastic vessels
    - “1D” = only varies along the length of the artery
    - “Nonlinear” = keeps nonlinear fluid effects
    - “Spectral element” = uses extremely accurate frequency-domain wave-based elements
    - “Viscoelastic” = artery wall includes damping and hysteresis
    - Apply the model to an example artery (test accuracy, efficiency, +  viscoelasticity and nonlinearities influence blood flow)



### Governing equations  

In general
- x = position along the artery
- t = time

Continuity Equation (mass conservation) : $\partial A/\partial t + \partial Q/\partial x = w$
- A(x,t) = cross-sectional area of the vessel (size of the open space inside)
- Q(x,t) = blood flow rate (volume per second passing through)
- w = leakage through the vessel wall (w = 0)
- whatever flows in minus whatever flows out = change in storage

Momentum Equation (Newton’s 2nd law): $\partial Q/\partial t + \partial (\gamma Q^2 A^{-1})/\partial x + \rho^{-1} A \partial P/\partial x = A f_x - \gamma Q A^{-1} + \mu \partial^2 Q/\partial x^2$

- Left-hand side = blood acceleration
    - $\partial Q/\partial t$: Change in flow rate over time
    - $\partial (\gamma Q^2 A^{-1})/\partial x$: Change in momentum due to velocity variations along the artery
    - $\rho^{-1} A \partial P/\partial x$: pressure forces speeding up or slowing down the blood (dirver of blood flow)

- Right-hand side = forces acting on blood
    - $A f_x$: Body forces (gravity, usually small)
    - $-\gamma Q/A$: Friction between the blood and the vessel wall
    - $\mu \partial^2 Q/\partial x^2$: Viscous effects (internal friction of blood)

Velocity Profile Parameters (c and γ)
- The velocity is fastest in the middle and slowest at the walls
- c = momentum correction factor
- γ = viscous/friction coefficient
- Values:
    - Uniform flow: γ=0, c=0
    - Parabolic flow (most realistic): γ=1/3, c=8π

Viscoelastic Constitutive Equation (Kelvin model)
- This is a stress–strain relationship for the vessel wall (Pressure difference (P – Pd)--> stress and Area change (A – Ad) → strain)
- The Kelvin model:
    - Elastic spring → responds instantly (stiffness)
    - Viscous dashpot → responds with delay (damping)
    - E = Young’s modulus (stiffness)
    - ν = Poisson’s ratio (thickness change coupling)
    - h = wall thickness
    - A_d, P_d = values at diastole
    - s_e, s_r = relaxation times
    - If pressure suddenly increases, the vessel:
    - expands immediately (elastic), then continues to adjust (viscous).

Simplify:
- Q_d, P_d, A_d = baseline values
- q, p, a = changes from baseline
- p=q=a=0 helps enforce boundary conditions and simplifies the equations

Slowly varying area assumption: $A_d(x) = A_0 (1 - hx)$
- This means the artery may taper (gradually narrow) but not sharply

FINAL GOVERNING EQATIONS !!!!!
- $u = [p, q]^T$ contains pressure and flow rate,
- The matrices M, C, G, D, A, K contain all physical parameters
- $f_e$ includes external effects (body force, leakage)
- $f_{NL}$ includes all nonlinear fluid terms

### Spectral element


First step --> convert everything from time domain --> frequency domain using Fourier series
- $u(x,t)$ (vector with $p$ and $q$), $f_e$, $f_{NL}$ all expanded as: sum of $\bar{u}(x,\omega_n) e^{i\omega_n t}$
- literally just discrete fourier transform
- after this substitution, the PDE becomes: $Du'' + Hu' + Lu = f_e + f_{NL}$, where $D,H,L$ depend on physical properties (viscosity, elasticity, etc)

- time derivatives become multiplications of $i \omega$
- so $\dot{u} → i \omega u$ and $\ddot{u} → -\omega^2 u$, now the whole thing is ODEs in $x$


Weak Form (variational form)
- multiply PDE by test function $du$ and integrate over element $[0,l]$
- integration by parts to handle $u''$ and $u'$ terms
- weak form is: $\int du'^T D u' \, dx - \int du'^T H u \, dx + \int du^T L u \, dx +$ boundary stuff $= \int du^T f_e \, dx - \int du^T f_{NL} \, dx$
- purpose: make it easier to put into element matrix form
- same idea as FEM but with frequency domain

Spectral Element DOFs
- a 1D artery element has two ends:
  - $x = 0$ (upstream)
  - $x = l$ (downstream)

- SEM nodal variables:
    - $dp = [ p(0),\ p(l) ]^T$
    - $dq = [ q(0),\ q(l) ]^T$

- these are the DOFs
- pressure and flow at the ends fully determine the solution inside (bc wave solutions)


Dynamic Shape Functions
- solve the homogeneous eqn: $Du'' + Hu' + Lu = 0$ and assume a wave solution: $u(x) = a e^{i k x}$
- sub this in --> dispersion relation: $b_1 k^2 + b_2 k + b_3 = 0$
- gives two wavenumbers $k_1$ and $k_2$
- so general solution is: $u(x) = c_1 e^{i k_1 x} + c_2 e^{i k_2 x}$
    - Each $k$ gives an eigenvector $a = [1,\ \phi]^T$ where $\phi$ comes from the eqn
    - match boundary values $(p_1, p_2)$ and $(q_1, q_2)$ to solve for $c_1, c_2$ then express: $u(x) = N_p(x, \omega)\, d_p$, where $N_p$ is the dynamic shape function matrix (depends on frequency, vessel props, wavenumbers, etc)

\*these shape functions are NOT polynomials(they are exact traveling-wave solutions)


Spectral Element Equation
- plug shape functions into weak form to get: $S(\omega)\, d_p = d_q + \hat{f}_e + \hat{f}_{NL}$, where: $S(\omega)$ = spectral element matrix (aka exact dynamic stiffness matrix)

$$\hat{f}_e = \int N_p^T f_e \, dx$$

$$\hat{f}_{NL} = \int N_p^T f_{NL} \, dx$$

- SEM version of FEM’s $K d = f$ but in frequency domain and with dynamic physics inside

- the $S(\omega)$ matrix is $2 \times 2$ because each element only has $p_1,p_2$

- they compute: $X(\omega) = \int N_p'^T D N_p' \, dx - \int N_p'^T H N_p \, dx + \int N_p^T L N_p \, dx$

        $Y(\omega) =$ boundary term matrix
        then: $S(\omega) = Y^{-1} X$


Assembly of Spectral Elements
- just like FEM:
  - assemble element $S(\omega)$ matrices into global $S_g$
  - assemble $d_q$ values
  - apply boundary conditions at inlet/outlet

global equation: $S_g(\omega)\, D_p = D_q + F_e + F_{NL}(p,q)$

Nonlinear part because $f_{NL}$ depends on $p$ and $q$, need iteration

**AFT (alternating frequency-time method)**:
   1. solve in freq domain
   2. convert to time using IFFT
   3. compute nonlinear $f_{NL}$ in time
   4. FFT it back to frequency
   5. iterate (until convergence)


### BRAINSTORM

Different approaches in paper:

1. Standard 1D FEM
   - The usual finite element method
   - Needs a lot of elements (100–400) to get wave accuracy

2. Frequency-domain SEM (the main method in the paper)
   - Uses exact wave solutions to build dynamic shape functions
   - Extremely accurate with very few elements (even one)

3. Time-domain SEM (from previous work, not this paper’s version) -- prob not
   - Polynomial interpolation in time
   - Authors basically say it’s inferior to the frequency-domain approach 

4.Linear vs nonlinear models
   - Linear: ignore nonlinear fluid terms
   - Nonlinear: keep convective and quadratic terms; requires the AFT (alternating frequency–time) iteration
   - Paper shows nonlinear effects are small but still important for completeness

5. Elastic vs viscoelastic wall models
   - Elastic: simple spring response
   - Viscoelastic: Kelvin model with damping and hysteresis
   - The viscoelastic behavior is a big focus of the paper

Maybe 
1. Frequency-domain SEM + linear + elastic
   - This is the simplest version of the framework
   - Lets us test the dynamic shape functions without the extra layers

2. Frequency-domain SEM + nonlinear + viscoelastic
   - full version used in the paper’s main results (Captures damping, hysteresis, and more realistic wave propagation)
   - Includes the AFT iteration and the Kelvin wall model